# Homework 5 Solution

**MECH.5130: Theory of Finite Element Analysis**

Michael N. Olaya

## Problem 1

![key_funcs](../figs/hw5_funcs.png)
![bubble_charts](../figs/hw5_bubble_charts.png)

## Problem 2

### compute_fs

```Python
# Input: 
    # funcs: ndarray (2 x 1)
    # coeff: ndarray (2 x ncoeff)
    # grid: ndarray (n x m x 2 x 1)
# Compute [f(x_1), f(x_2)] for all grid points and coefficients
# Output: 
    # ndarray (n x m x 2 x 1)
```

### compute_J_det_surf

```Python
# Input: 
    # funcs: ndarray (2 x 1)
    # grid_shape: ndarray (4 x 0)
    # face: str (+x, -x, +y, -y)
# Get J1, J2 component indices associated with load face for computing det(J) on the surface
# Compute det(J) on the surface = (J1**2 + J2**2)**0.5
# Output: 
    # ndarray (n x m x 1 x 1)
```

### compute_force_vector

```Python
# Input: 
    # ip_grid: ndarray (num_pts x num_pts x 2 x 1)
    # w_ij: ndarray (num_pts x num_pts x 1 x 1)
    # face: str (+x, -x, +y, -y)
    # thickness: float
    # funcs: ndarray of Callable (2 x 1)
    # coeff: ndarray (2 x ncoeff)
    # elem: mfe.baseclasses.Element2D
# Compute [N] and [N]^T for the element
# Compute [f^s] for the element
# Compute [dN]
# Compute [J]
# Compute det(J) on the surface
# Compute f on the surface = thickness*w_ij*det(J_surf)*[N]^T[f^s]
# Output: 
    # ndarray (n x m x 2*nnodes x 1)
```

## Problem 3

### Force vector derivation

Assuming no body forces ($[{f}^{b}] = [0]$)

$$
\begin{align*}
[{f}] & = \int_{\Gamma} [{N}]^{T} [{f}^{s}] d\Gamma \\
& = t_{z} \int_{y} [{N}]^{T} [{f}^{s}] dy
\end{align*}
$$

Surface traction is along the $\eta_{1} = 1$ in isoparametric space, thus

$$
\begin{align*}
[{f}] & = t_{z} \int_{y} [{N}]^{T} [{f}]^{s} dy \\
& = t_{z}*\det([J^{\Gamma}])* \int_{-1}^{1} [{N}]^{T} [{f}]^{s} d\eta_{2}
\end{align*}
$$

Evaluating shape functions @ $\eta_{1} = 1$

$$
\begin{align*}
N_{1} & = 0.25*(\eta_{1} - 1)*(\eta_{2} - 1) \rightarrow 0.25*(1 - 1)*(\eta_{2} - 1) = 0 \\
N_{2} & = -0.25*(\eta_{1} + 1)*(\eta_{2} - 1) \rightarrow -0.25*(1 + 1)*(\eta_{2} - 1) = -0.5*(\eta_{2} - 1) \\
N_{3} & = 0.25*(\eta_{1} + 1)*(\eta_{2} + 1) \rightarrow 0.25*(1 + 1)*(\eta_{2} + 1) = 0.5*(\eta_{2} + 1) \\
N_{4} & = -0.25*(\eta_{1} - 1)*(\eta_{2} + 1) \rightarrow -0.25*(1 - 1)*(\eta_{2} + 1) = 0 \\
\end{align*}
$$

Assembling into the shape function matrix:

$$
[N] = \begin{bmatrix}
    N_{1} & 0 & N_{2} & 0 & N_{3} & 0 & N_{4} & 0 \\
    0 & N_{1} & 0 & N_{2} & 0 & N_{3} & 0 & N_{4}
\end{bmatrix} = \begin{bmatrix}
    0 & 0 & -0.5*(\eta_{2} - 1) & 0 & 0.5*(\eta_{2} + 1) & 0 & 0 & 0 \\
    0 & 0 & 0 & -0.5*(\eta_{2} - 1) & 0 & 0.5*(\eta_{2} + 1) & 0 & 0
\end{bmatrix}
$$

For the given surface traction, $\det([J^{\Gamma}])$ requires components Jacobian components $J_{21}, J_{22}$. Hence, from the equation for the Jacobian in column format, only rows 2 and 4 of the shape function derivative matrix are required to compute $\det([J^{\Gamma}])$. Therefore, only the derivatives with respect to $\eta_2$ of the shape functions can be inspected here.

Evaluating shape function derivatives @ $\eta_{1} = 1$

$$
\begin{align*}
\frac{\partial{N_{1}}}{\partial{\eta_{2}}} & = 0.25*(\eta_{1} - 1) \rightarrow 0.25*(1 - 1) = 0\\
\frac{\partial{N_{2}}}{\partial{\eta_{2}}} & = -0.25*(\eta_{1} + 1) \rightarrow -0.25*(1 + 1) = -0.5\\
\frac{\partial{N_{3}}}{\partial{\eta_{2}}} & = 0.25*(\eta_{1} + 1) \rightarrow 0.25*(1 + 1) = 0.5 \\
\frac{\partial{N_{4}}}{\partial{\eta_{2}}} & = -0.25*(\eta_{1} - 1) \rightarrow -0.25*(1 - 1) = 0 \\
\end{align*}
$$

Now computing Jacobian components $J_{21}$ and $J_{22}$:

$$
[J_{col}] = \frac{\partial[{N}]}{\partial{\mathbf{\eta}}}{[\hat{x}]}
$$

$$
J_{21} = \frac{\partial{N_{1}}}{\partial{\eta_{2}}}*\hat{x_{11}} + \frac{\partial{N_{1}}}{\partial{\eta_{2}}}*\hat{x_{21}} + \frac{\partial{N_{1}}}{\partial{\eta_{2}}}*\hat{x_{31}} + \frac{\partial{N_{1}}}{\partial{\eta_{2}}}*\hat{x_{41}} = -0.5*\hat{x_{21}} + 0.5*\hat{x_{31}}
$$

$$
J_{22} = \frac{\partial{N_{1}}}{\partial{\eta_{2}}}*\hat{x_{12}} + \frac{\partial{N_{1}}}{\partial{\eta_{2}}}*\hat{x_{22}} + \frac{\partial{N_{1}}}{\partial{\eta_{2}}}*\hat{x_{32}} + \frac{\partial{N_{1}}}{\partial{\eta_{2}}}*\hat{x_{42}} = -0.5*\hat{x_{22}} + 0.5*\hat{x_{32}}
$$

And finally $\det([J^{\Gamma}])$ on the surface:

$$
\begin{align*}
\det([J^{\Gamma}]) & = (J_{21}^{2} + J_{22}^{2})^{0.5} = ((-0.5*\hat{x_{21}} + 0.5*\hat{x_{31}})^{2} + (-0.5*\hat{x_{22}} + 0.5*\hat{x_{32}})^{2})^{0.5} \\
\end{align*}
$$

With all components assembled, for a given thickness $t_{z}$, compute $[f]$, where $w_{ij}$ are the integration point weights along the load face.
$$
[{f}] = t_{z}*\det([J^{\Gamma}])*w_{ij}*[{N}]^{T} [{f}^{s}]
$$

In [59]:
import numpy as np
import matplotlib.pyplot as plt

import mfe.utils
import mfe.elem_lib
import mfe.baseclasses
import mfe.gauss
import mfe.plot
import mfe.load

# plt.style.use('ggplot')
# plt.style.use('../../mfe/def_plt_style.mplstyle')

For a linear 2D element, the integral has order 1 in each coordinate direction. Therefore, the minimum number of integration points $n$ must be 2 as shown below:

$$
(2n - 1) \ge 2 \\
\therefore n \ge 1.5
$$

In [60]:
# Create element and surface traction
elem = mfe.elem_lib.Linear2D.from_element_coords(
    [
        np.array([0, 0]), 
        np.array([12, -1]), 
        np.array([15, 8]), 
        np.array([-1, 10])
    ], num_pts=2
)
bc = mfe.load.SurfaceTraction.generate(elem=elem, face='+x', constants=[np.array([4, 3]), np.array([5, 1])], thickness=1.3)
f = bc.compute_force_vector(elem)
print(f)

[[  0.        ]
 [  0.        ]
 [265.15699043]
 [ 43.16509869]
 [283.65629749]
 [ 61.66440575]
 [  0.        ]
 [  0.        ]]


## Problem 4

For a linear 2D element, the integral has order 2 in each coordinate direction. Therefore, the minimum number of integration points $n$ must be 3 as shown below:

$$
(2n - 1) \ge 4 \\
\therefore n \ge 2.5
$$

In [61]:
# Create element and surface traction
elem = mfe.elem_lib.Quadratic2D.from_element_coords(
    [
        np.array([0, 0]), 
        np.array([6, -0.5]), 
        np.array([12, -1]), 
        np.array([11, 3]),
        np.array([15, 8]), 
        np.array([6, 11]), 
        np.array([-1, 10]), 
        np.array([2, 5]),
    ], num_pts=3
)
bc = mfe.load.SurfaceTraction.generate(elem=elem, face='+x', constants=[np.array([4, 3]), np.array([5, 1])], thickness=1.3)
f = bc.compute_force_vector(elem)
print(f)

[[  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [ 61.99153134]
 [  4.96061823]
 [349.16398252]
 [ 75.62878973]
 [157.54376222]
 [ 42.70728459]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]]
